## Configuração inicial

In [1]:
import pickle
import pandas as pd

from Utils import templates, utils, recommender

In [ ]:
## Configurações base

config = {
    #"runtime": "ROCm llama.cpp v1.23.0", 
    #"runtime": "CPU llama.cpp v1.22.2", # performance ruim
    "LLM_runtime": "Vulkan llama.cpp v1.23.0", # melhor opção
    "dataset": "ml_100k",
    "nsu" : 12,     # número de usuários para filtragem colaborativa        | SSBD :12  | Default :18   |            | Best = 19
    "nci" :19,      # número de itens para filtragem colaborativa           | SSBD :19  | Default :24   | Max : 1682 |
    "lenlimit" : 8,  # limite de tamanho para a lista filmes assistidos     | SSBD : 8  | Default :24   | Max : 1682 | Best = 8
    "test_run" : 0, # define a quantidade de recomendações,                 |           | Default :0    | Max : 943 
    "obs": "teste prompt3 novo"
}

## define o prompt template
prompt_template = templates.PROMPT_TEMPLATE_2
config.update({"prompt_template": prompt_template})

## define o prompt para formatar a resposta final 
#prompt_format = templates.PROMPT_TEMPLATE_ESTRUCTURE
prompt_format = "" # para não utilizar 
config.update({"prompt_format": prompt_format})

# load movie lens 100k dataset
dataset = utils.read_json("Data/ML100K_clean.json")
print(f'Quantidade de Usuários: {len(dataset)}')

Quantidade de Usuários: 943


---------   
## Escolha do modelo

### llama 3 finetunado

In [3]:
config.update({
    "model_name" :"llama3_fn",
    "Arch" : "llama",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.0,
    "max_tokens" : -1,  # Default : 4096
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False, # não vi vantagem no uso 
})

### mistral-7b-instruct-v0.3

In [ ]:
config.update({
    "model_name" :"mistral-7b-instruct-v0.3",
    "Arch" : "llama",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : -1,  # Default : 4096
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False, # não vi vantagem no uso 
})

### gemma-3-4b-it

In [ ]:
config.update({
    "model_name" :"gemma-3-4b-it",
    "Arch" : "gemma3",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : -1,  # Default : 4096
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False, # não vi vantagem no uso 
})

### meta-llama-3.1-8b-instruct

In [ ]:
config.update({
    "model_name" :"meta-llama-3.1-8b-instruct",
    "Arch" : "llama",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : 4096,
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False,
})

### llama-3.2-3b-instruct

In [ ]:
config.update({
    "model_name" :"llama-3.2-3b-instruct",
    "Arch" : "llama",
    "Quantization" : "Q8_0",
    "Temperature": 0,
    "max_tokens" : -1, #Default :4096
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False,
})

In [ ]:
# retorna apenas o usuários que tem o gt no candidatos.
# implementar sobre o pipeline do projeto 

def get_candidate_ids_list(data, id_list, user_matrix_sim, num_u, num_i):
    cand_ids = []
    for i in id_list:
        watched_movies = data[i][0].split(' | ')
        candidate_items = utils.sort_user_filtering_items(data, watched_movies, user_matrix_sim[i], num_u, num_i)
        if data[i][-1] in candidate_items:
            cand_ids.append(i)
    return cand_ids

In [ ]:
id_list = list(range(0, len(dataset)))
#assert(len(id_list) == 943) # aqui é verificado se a lista possue exatamente essa quantidade

# Building indexes and similarity matrices for users and movies.
movie_idx = utils.build_moviename_index_dict(dataset)
user_sim_matrix = utils.build_user_similarity_matrix(dataset, movie_idx)


# para fazer a filtragem sobre os filmes 
#pop_dict = utils.build_movie_popularity_dict(dataset) 
#item_sim_matrix = utils.build_item_similarity_matrix(dataset)

## Execução

In [4]:
result_pkl = recommender.recommendation_workflow(config         = config,
                                                 dataset        = dataset,
                                                 prompt_template= prompt_template,
                                                 prompt_format  = prompt_format)

Processando:   0%|          | 0/943 [00:00<?, ?it/s]

## execução varias configs

In [ ]:
import pickle
import pandas as pd

from Utils import templates, utils, recommender

## PROMPT 2 

config = {}

## define o prompt template
prompt_template = templates.PROMPT_TEMPLATE_2
config.update({"prompt_template": prompt_template})

## define o prompt para formatar a resposta final 
#prompt_format = templates.PROMPT_TEMPLATE_ESTRUCTURE
prompt_format = "" # para não utilizar 
config.update({"prompt_format": prompt_format})

# load movie lens 100k dataset
dataset = utils.read_json("Data/ML100K_clean.json")
print(f'Quantidade de Usuários: {len(dataset)}')

config.update({
    #"runtime": "ROCm llama.cpp v1.23.0", 
    #"runtime": "CPU llama.cpp v1.22.2", # performance ruim
    "LLM_runtime": "Vulkan llama.cpp v1.23.0", # melhor opção
    "dataset": "ml_100k",
    "nsu" : 19,     # número de usuários para filtragem colaborativa        | SSBD :12  | Default :18   | 
    "nci" :19,      # número de itens para filtragem colaborativa           | SSBD :19  | Default :24   | Max : 1682
    "lenlimit" : 8,  # limite de tamanho para a lista filmes assistidos    | SSBD : 8  | Default :24   | Max : 1682
    "test_run" : 0, # define a quantidade de recomendações,                 |           | Default :0    | Max : 943 
    "obs": "testando prompt novo - prompt2"
})

config.update({
    "model_name" :"gemma-3-4b-it",
    "Arch" : "gemma3",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : -1,  # Default : 4096
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False, # não vi vantagem no uso 
})

config_list = []

config1 = config.copy()
config_list.append(config1)

for i in range(0, len(config_list)):
    config = config_list[i]
    print(f'Rodando configuração {i} de {len(config_list)}')
    try:
        result_pkl = recommender.recommendation_workflow(config         = config,
                                                         dataset        = dataset,
                                                        prompt_template= prompt_template,
                                                        prompt_format  = prompt_format)
        print(f'Configuração {i} de {len(config_list)} finalizada')
    except Exception as e:
        print(f'Erro na configuração {i} de {len(config_list)}')
        print(e)
        continue

## PROMPT 3 

config = {}

## define o prompt template
prompt_template = templates.PROMPT_TEMPLATE_3
config.update({"prompt_template": prompt_template})

## define o prompt para formatar a resposta final 
#prompt_format = templates.PROMPT_TEMPLATE_ESTRUCTURE
prompt_format = "" # para não utilizar 
config.update({"prompt_format": prompt_format})

# load movie lens 100k dataset
dataset = utils.read_json("Data/ML100K_clean.json")
print(f'Quantidade de Usuários: {len(dataset)}')

config.update({
    #"runtime": "ROCm llama.cpp v1.23.0", 
    #"runtime": "CPU llama.cpp v1.22.2", # performance ruim
    "LLM_runtime": "Vulkan llama.cpp v1.23.0", # melhor opção
    "dataset": "ml_100k",
    "nsu" : 12,     # número de usuários para filtragem colaborativa        | SSBD :12  | Default :18   | 
    "nci" :19,      # número de itens para filtragem colaborativa           | SSBD :19  | Default :24   | Max : 1682
    "lenlimit" : 8,  # limite de tamanho para a lista filmes assistidos    | SSBD : 8  | Default :24   | Max : 1682
    "test_run" : 0, # define a quantidade de recomendações,                 |           | Default :0    | Max : 943 
    "obs": "testando prompt novo - prompt3"
})

config.update({
    "model_name" :"gemma-3-4b-it",
    "Arch" : "gemma3",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : -1,  # Default : 4096
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False, # não vi vantagem no uso 
})

config_list = []

config1 = config.copy()
config_list.append(config1)

for i in range(0, len(config_list)):
    config = config_list[i]
    print(f'Rodando configuração {i} de {len(config_list)}')
    try:
        result_pkl = recommender.recommendation_workflow(config         = config,
                                                         dataset        = dataset,
                                                        prompt_template= prompt_template,
                                                        prompt_format  = prompt_format)
        print(f'Configuração {i} de {len(config_list)} finalizada')
    except Exception as e:
        print(f'Erro na configuração {i} de {len(config_list)}')
        print(e)
        continue

Quantidade de Usuários: 943
Rodando configuração 0 de 1


Processando:   0%|          | 0/943 [00:00<?, ?it/s]

Configuração 0 de 1 finalizada
Quantidade de Usuários: 943
Rodando configuração 0 de 1


Processando:   0%|          | 0/943 [00:00<?, ?it/s]

Configuração 0 de 1 finalizada


In [ ]:
for i in range(0, len(config_list)):
    config = config_list[i]
    print(config)

## Resultados

In [ ]:
with open(f'{result_pkl}', 'rb') as f:
    data = pickle.load(f)

In [ ]:
results = []
for key, value in data.items():
    if isinstance(key, int) and isinstance(value, dict):  # Pegando apenas os experimentos
        results.append({
            'Candidates': value.get('candidate_set', ''),
            'Ground Truth': value.get('ground_truth', ''),
            'gt_in_candidate_set': value.get('gt_in_candidate_set', ''),
            #'Input 1': value.get('input_1', ''),
            #'Predictions 1': value.get('predictions_1', ''),
            #'Input 2': value.get('input_2', ''),
            #'Predictions 2': value.get('predictions_2', ''),
            'Input 3': value.get('input_3', ''),
            'Predictions 3': value.get('predictions_3', ''),
            #'Recommendations': value.get('recommendations', ''),
            'rec_HitRate@10': value.get('rec_HitRate@10', ''),
            #'Precision': value.get('precision', ''),
            #'Recall': value.get('recall', ''),
            'rec_NDCG@10': value.get('rec_NDCG@10', '')
        })

df_results = pd.DataFrame(results)

df_results

# Preparar dataset

In [1]:
import random
import pandas as pd
from Utils import templates, utils

nsu = 12
nci = 19

dataset = utils.read_json("Data/ML100K_clean.json")
movie_idx = utils.build_moviename_index_dict(dataset)
user_sim_matrix = utils.build_user_similarity_matrix(dataset, movie_idx)
id_list = list(range(0, len(dataset)))

data_list = []
id_list_com_gt_no_candidate=[]

for i in id_list:

  watched_mv = dataset[i][0].split(' | ')[::-1]
  groundTruth = dataset[i][-1]
  candidate_items = utils.sort_collaborative_user_filtering(target_user_id=i,
                                                                    dataset=dataset,
                                                                    user_similarity_matrix=user_sim_matrix,
                                                                    num_users=nsu,
                                                                    num_items=nci,
                                                                    include_similar_user_GT=False,
                                                                    debug=False)
  random.shuffle(candidate_items)

  # verifica se o ground_truth está no candidate_set
  gt_in_candidate_set = True if any(groundTruth.lower() in candidate.lower() for candidate in candidate_items) else False

  if gt_in_candidate_set == True:
    id_list_com_gt_no_candidate.append(i)

  data_list.append({
        'user_id': i,
        'watched_movies': watched_mv,
        'ground_truth': groundTruth,
        'candidate_items': candidate_items,
        'gt_in_candidate_set': gt_in_candidate_set
  })

df = pd.DataFrame(data_list)

print(f'nsu: {nsu} \nnci: {nci} \nQnt de usuários do dataset: {len(dataset)} \nQnt de usuários com gt no candidate: {len(id_list_com_gt_no_candidate)}')

nsu: 12 
nci: 19 
Qnt de usuários do dataset: 943 
Qnt de usuários com gt no candidate: 152


In [8]:
# Define a proporção de divisão
train_ratio = 0.8
test_ratio = 0.2

# Embaralha os IDs para garantir aleatoriedade
random.shuffle(id_list_com_gt_no_candidate)

# Calcula o tamanho de cada conjunto
total_ids = len(id_list_com_gt_no_candidate)
train_size = int(train_ratio * total_ids)
test_size = total_ids - train_size

# Separa os IDs em conjuntos
train_ids = id_list_com_gt_no_candidate[:train_size]
test_ids = id_list_com_gt_no_candidate[train_size:]

# Imprime o tamanho de cada conjunto para verificação
print(f"Tamanho do conjunto de treino: {len(train_ids)}")
print(f"Tamanho do conjunto de teste: {len(test_ids)}")

train_dataset = []

# configs 
model_name = "gemma-3-4b-it"
temperature = 0
prompt_template = templates.PROMPT_TEMPLATE_2
system_prompt = "YYou are a movie expert provide the answer for the question based on the given context. Your answer must be short " 
max_tokens = -1
lenlimit = 8

for id in train_ids:
    filtered = df[df['user_id'] == id]
    if not filtered.empty:
        candidate_items = filtered['candidate_items'].iloc[0]
        ground_truth = filtered['ground_truth'].iloc[0]
        watched_movies = filtered['watched_movies'].iloc[0]
    else:
        continue 

    input_prompt = prompt_template['Preference'].format(', '.join(candidate_items),', '.join(watched_movies))
    response = utils.query_lm_studio(model_name,0.0,system_prompt,input_prompt,max_tokens)
    movie_preference = response

    input_prompt = prompt_template['Featured_movies'].format(', '.join(watched_mv[-lenlimit:]), movie_preference)
    response = utils.query_lm_studio(model_name,0.0,system_prompt,input_prompt,max_tokens)
    most_featured = response

    input = templates.TRAIN_TEMPLATE['INPUT'].format(', '.join(candidate_items),', '.join(watched_movies),movie_preference,most_featured )
    output= templates.TRAIN_TEMPLATE['OUTPUT'].format(ground_truth)

    train_dataset.append({
        'input': input,
        'output': output
  })



Tamanho do conjunto de treino: 121
Tamanho do conjunto de teste: 31


In [ ]:
df_train = pd.DataFrame(train_dataset)
df_train



,input,output
0,\n\n ### CANDIDATE MOVIE SET: Starship Troo...,1.Chasing Amy
1,\n\n ### CANDIDATE MOVIE SET: Absolute Powe...,1.Cape Fear
2,"\n\n ### CANDIDATE MOVIE SET: Eraser, Volca...",1.Jerry Maguire
3,"\n\n ### CANDIDATE MOVIE SET: Phenomenon, E...",1.River Wild
4,"\n\n ### CANDIDATE MOVIE SET: Omen, 20,000 ...",1.Grifters
...,...,...
116,\n\n ### CANDIDATE MOVIE SET: L.A. Confiden...,1.Conspiracy Theory
117,\n\n ### CANDIDATE MOVIE SET: Starship Troo...,1.Full Monty
118,"\n\n ### CANDIDATE MOVIE SET: Jaws, Godfath...",1.Godfather: Part II
119,"\n\n ### CANDIDATE MOVIE SET: Godfather, Fa...",1.Silence of the Lambs


In [10]:
df_train.to_pickle('Data/ML100K_train.pkl')

In [12]:
test_dataset = []

for id in test_ids:
    filtered = df[df['user_id'] == id]
    if not filtered.empty:
        candidate_items = filtered['candidate_items'].iloc[0]
        ground_truth = filtered['ground_truth'].iloc[0]
        watched_movies = filtered['watched_movies'].iloc[0]
    else:
        continue 
    
    test_dataset.append({
        'user_id': id,
        'candidate_items': candidate_items,
        'ground_truth': ground_truth,
        'watched_movies': watched_movies
    })
df_test = pd.DataFrame(test_dataset)
df_test.to_pickle('Data/ML100K_test.pkl')
df_test

,user_id,candidate_items,ground_truth,watched_movies
0,507,"[Right Stuff, Braveheart, GoodFellas, Platoon,...",Star Trek: The Wrath of Khan,"[Star Wars, Miracle on 34th Street, E.T. the E..."
1,210,"[Jerry Maguire, Back to the Future, Contact, R...",Return of the Jedi,"[Rock, Jackie Chan's First Strike, Men in Blac..."
2,47,"[Glory, Stand by Me, Schindler's List, Godfath...",Silence of the Lambs,"[L.A. Confidential, Amadeus, This Is Spinal Ta..."
3,365,"[Chasing Amy, Clockwork Orange, Tales From the...",Candyman,"[Howling, Mute Witness, Mary Shelley's Franken..."
4,426,"[George of the Jungle, Evita, Mother, Liar Lia...",Evita,"[U Turn, Anna Karenina, Wishmaster, Thousand A..."
5,601,"[Truth About Cats & Dogs, Courage Under Fire, ...",Twister,"[Rock, Phenomenon, People vs. Larry Flynt, Men..."
6,17,"[Omen, Lawrence of Arabia, Hunt for Red Octobe...",Shawshank Redemption,"[Three Colors: Red, Raise the Red Lantern, Thi..."
7,919,"[Eve's Bayou, Volcano, Apt Pupil, Alien: Resur...",Chasing Amy,"[Hoodlum, Devil's Own, Wag the Dog, Contact, E..."
8,198,"[Big Night, Jerry Maguire, Truth About Cats & ...",Truth About Cats & Dogs,"[Postino, Il, Mission: Impossible, James and t..."
9,579,"[Jerry Maguire, Mars Attacks!, Multiplicity, R...",River Wild,"[Fled, Willy Wonka and the Chocolate Factory, ..."
